In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
def build_base(inputSize, loss_function = "mse", optimizer = "adam"):
    model = Sequential()
    model.add(Dense(inputSize[0]//2, input_shape = inputSize, activation = "relu"))
    model.add(Dense(inputSize[0], activation = "relu"))
    model.compile(loss = loss_function, optimizer = optimizer, metrics = ["mse"])

    return model

def get_encoded_data(model, decoder_layers, xTrain, loss_function = "mse", optimizer = "adam"):
    decoder_layers.append(model.layers[-1])
    model.pop()
    model.compile(loss = loss_function, optimizer = optimizer, metrics = [loss_function])
    model.summary()
    print()
    xTrainEncoded = model.predict(xTrain)
    print(xTrainEncoded.shape)
    
    return xTrainEncoded, decoder_layers

def add_layer(model, layer_dimension):
    for layer in model.layers:
        layer.trainable = False
    model.add(Dense(layer_dimension, activation = "relu"))
    model.add(Dense(layer_dimension*2, activation = "relu"))
    model.compile(loss = "mse", optimizer = "adam", metrics = ["mse"])
    model.summary()
    
    return model

# 1st Autoencoder

In [4]:
model = build_base((32*32,), "mse", "adam")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
Total params: 1,050,112
Trainable params: 1,050,112
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Treinamos a primeiro autoencoder

xTrain = np.load("../data/711_input.npy").astype("float32")

model.fit(xTrain, xTrain, batch_size = 128, epochs = 100, verbose = 1)

Train on 10499 samples
Epoch 1/100
10499/10499 [==============================] - 2s 226us/sample - loss: 0.0326 - mse: 0.0326
Epoch 2/100
10499/10499 [==============================] - 1s 48us/sample - loss: 0.0303 - mse: 0.0303
Epoch 3/100
10499/10499 [==============================] - 0s 46us/sample - loss: 0.0303 - mse: 0.0303
Epoch 4/100
10499/10499 [==============================] - 1s 49us/sample - loss: 0.0302 - mse: 0.0302
Epoch 5/100
10499/10499 [==============================] - 1s 49us/sample - loss: 0.0302 - mse: 0.0302
Epoch 6/100
10499/10499 [==============================] - 0s 47us/sample - loss: 0.0302 - mse: 0.0302
Epoch 7/100
10499/10499 [==============================] - 0s 47us/sample - loss: 0.0302 - mse: 0.0302
Epoch 8/100
10499/10499 [==============================] - 1s 51us/sample - loss: 0.0302 - mse: 0.0302
Epoch 9/100
10499/10499 [==============================] - 1s 50us/sample - loss: 0.0302 - mse: 0.0302
Epoch 10/100
10499/10499 [=======================

10499/10499 [==============================] - 1s 73us/sample - loss: 0.0298 - mse: 0.0298
Epoch 80/100
10499/10499 [==============================] - 1s 48us/sample - loss: 0.0298 - mse: 0.0298
Epoch 81/100
10499/10499 [==============================] - 1s 121us/sample - loss: 0.0298 - mse: 0.0298
Epoch 82/100
10499/10499 [==============================] - 1s 119us/sample - loss: 0.0298 - mse: 0.0298
Epoch 83/100
10499/10499 [==============================] - 1s 67us/sample - loss: 0.0298 - mse: 0.0298
Epoch 84/100
10499/10499 [==============================] - 1s 112us/sample - loss: 0.0298 - mse: 0.0298
Epoch 85/100
10499/10499 [==============================] - 1s 51us/sample - loss: 0.0298 - mse: 0.0298
Epoch 86/100
10499/10499 [==============================] - 1s 105us/sample - loss: 0.0298 - mse: 0.0298
Epoch 87/100
10499/10499 [==============================] - 1s 62us/sample - loss: 0.0298 - mse: 0.0298
Epoch 88/100
10499/10499 [==============================] - 1s 89us/sampl

# 2nd Autoencoder

In [6]:
# Retiramos a layer de decodificação, e pegamos os dados codificados para treinar o próximo autoencoder.

xTrainEncoded, decoder_layers = get_encoded_data(model, [], xTrain)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
Total params: 524,800
Trainable params: 524,800
Non-trainable params: 0
_________________________________________________________________

(10499, 512)


In [7]:
# Adicionando o segundo autoencoder

model = add_layer(model, 256)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               131584    
Total params: 787,712
Trainable params: 262,912
Non-trainable params: 524,800
_________________________________________________________________


In [8]:
# Treinando o segundo autoencoder

model.fit(xTrain, xTrainEncoded, batch_size = 128, epochs = 100, verbose = 1)

Train on 10499 samples
Epoch 1/100
10499/10499 [==============================] - 1s 56us/sample - loss: 0.0130 - mse: 0.0130
Epoch 2/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0089 - mse: 0.0089
Epoch 3/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0075 - mse: 0.0075
Epoch 4/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0073 - mse: 0.0073
Epoch 5/100
10499/10499 [==============================] - 0s 37us/sample - loss: 0.0066 - mse: 0.0066
Epoch 6/100
10499/10499 [==============================] - 0s 37us/sample - loss: 0.0066 - mse: 0.0066
Epoch 7/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0065 - mse: 0.0065
Epoch 8/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0065 - mse: 0.0065
Epoch 9/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0065 - mse: 0.0065
Epoch 10/100
10499/10499 [========================

10499/10499 [==============================] - 0s 35us/sample - loss: 0.0061 - mse: 0.0061
Epoch 80/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0061 - mse: 0.0061
Epoch 81/100
10499/10499 [==============================] - 0s 35us/sample - loss: 0.0061 - mse: 0.0061
Epoch 82/100
10499/10499 [==============================] - 0s 36us/sample - loss: 0.0061 - mse: 0.0061
Epoch 83/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0061 - mse: 0.0061
Epoch 84/100
10499/10499 [==============================] - 0s 35us/sample - loss: 0.0061 - mse: 0.0061
Epoch 85/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0061 - mse: 0.0061
Epoch 86/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0061 - mse: 0.0061
Epoch 87/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0061 - mse: 0.0061
Epoch 88/100
10499/10499 [==============================] - 0s 36us/sample - 

# 3rd Autoencoder

In [9]:
# Training 3rd autoencoder

xTrainEncoded, decoder_layers = get_encoded_data(model, decoder_layers, xTrain)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
Total params: 656,128
Trainable params: 131,328
Non-trainable params: 524,800
_________________________________________________________________

(10499, 256)


In [10]:
model = add_layer(model, 128)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 256)               33024     
Total params: 722,048
Trainable params: 65,920
Non-trainable params: 656,128
_________________________________________________________________


In [11]:
model.fit(xTrain, xTrainEncoded, batch_size = 128, epochs = 100, verbose = 1)

Train on 10499 samples
Epoch 1/100
10499/10499 [==============================] - 1s 73us/sample - loss: 0.0125 - mse: 0.0125
Epoch 2/100
10499/10499 [==============================] - 0s 44us/sample - loss: 0.0060 - mse: 0.0060
Epoch 3/100
10499/10499 [==============================] - 0s 35us/sample - loss: 0.0045 - mse: 0.0045
Epoch 4/100
10499/10499 [==============================] - 0s 40us/sample - loss: 0.0034 - mse: 0.0034
Epoch 5/100
10499/10499 [==============================] - 0s 44us/sample - loss: 0.0033 - mse: 0.0033
Epoch 6/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0032 - mse: 0.0032
Epoch 7/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0032 - mse: 0.0032
Epoch 8/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0030 - mse: 0.0030
Epoch 9/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0022 - mse: 0.0022
Epoch 10/100
10499/10499 [========================

10499/10499 [==============================] - 0s 40us/sample - loss: 0.0018 - mse: 0.0018
Epoch 80/100
10499/10499 [==============================] - 0s 38us/sample - loss: 0.0018 - mse: 0.0018
Epoch 81/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0018 - mse: 0.0018
Epoch 82/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0018 - mse: 0.0018
Epoch 83/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0018 - mse: 0.0018
Epoch 84/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0018 - mse: 0.0018
Epoch 85/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0018 - mse: 0.0018
Epoch 86/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0018 - mse: 0.0018
Epoch 87/100
10499/10499 [==============================] - 0s 38us/sample - loss: 0.0018 - mse: 0.0018
Epoch 88/100
10499/10499 [==============================] - 0s 31us/sample - 

# 4th Autoencoder

In [12]:
xTrainEncoded, decoder_layers = get_encoded_data(model, decoder_layers, xTrain)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
Total params: 689,024
Trainable params: 32,896
Non-trainable params: 656,128
_________________________________________________________________

(10499, 128)


In [13]:
model = add_layer(model, 64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
Total params: 705,600
Trainable params: 16,576
Non-trainable params: 689,024
_________________________________________________________________


In [14]:
model.fit(xTrain, xTrainEncoded, batch_size = 128, epochs = 100, verbose = 1)

Train on 10499 samples
Epoch 1/100
10499/10499 [==============================] - 1s 69us/sample - loss: 0.0188 - mse: 0.0188
Epoch 2/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0095 - mse: 0.0095
Epoch 3/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0075 - mse: 0.0075
Epoch 4/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0069 - mse: 0.0069
Epoch 5/100
10499/10499 [==============================] - 0s 32us/sample - loss: 0.0066 - mse: 0.0066
Epoch 6/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0063 - mse: 0.0063
Epoch 7/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0062 - mse: 0.0062
Epoch 8/100
10499/10499 [==============================] - 0s 31us/sample - loss: 0.0061 - mse: 0.0061
Epoch 9/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0061 - mse: 0.0061
Epoch 10/100
10499/10499 [========================

10499/10499 [==============================] - 0s 42us/sample - loss: 0.0057 - mse: 0.0057
Epoch 80/100
10499/10499 [==============================] - 0s 37us/sample - loss: 0.0057 - mse: 0.0057
Epoch 81/100
10499/10499 [==============================] - 0s 36us/sample - loss: 0.0057 - mse: 0.0057
Epoch 82/100
10499/10499 [==============================] - 0s 44us/sample - loss: 0.0057 - mse: 0.0057
Epoch 83/100
10499/10499 [==============================] - 0s 35us/sample - loss: 0.0057 - mse: 0.0057
Epoch 84/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0057 - mse: 0.0057
Epoch 85/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0057 - mse: 0.0057
Epoch 86/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0057 - mse: 0.0057
Epoch 87/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0057 - mse: 0.0057
Epoch 88/100
10499/10499 [==============================] - 0s 37us/sample - 

# 5th Autoencoder

In [15]:
xTrainEncoded, decoder_layers = get_encoded_data(model, decoder_layers, xTrain)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
Total params: 697,280
Trainable params: 8,256
Non-trainable params: 689,024
_________________________________________________________________

(10499, 64)


In [16]:
model = add_layer(model, 32)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2112      
Total params: 701,472
Trainable params: 4,192
Non-trainable params: 697,280
______________________________________________

In [17]:
model.fit(xTrain, xTrainEncoded, batch_size = 128, epochs = 100, verbose = 1)

Train on 10499 samples
Epoch 1/100
10499/10499 [==============================] - 1s 70us/sample - loss: 0.0564 - mse: 0.0564
Epoch 2/100
10499/10499 [==============================] - 0s 40us/sample - loss: 0.0331 - mse: 0.0331
Epoch 3/100
10499/10499 [==============================] - 0s 44us/sample - loss: 0.0313 - mse: 0.0313
Epoch 4/100
10499/10499 [==============================] - 0s 39us/sample - loss: 0.0287 - mse: 0.0287
Epoch 5/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0268 - mse: 0.0268
Epoch 6/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0263 - mse: 0.0263
Epoch 7/100
10499/10499 [==============================] - 0s 45us/sample - loss: 0.0260 - mse: 0.0260
Epoch 8/100
10499/10499 [==============================] - 0s 45us/sample - loss: 0.0257 - mse: 0.0257
Epoch 9/100
10499/10499 [==============================] - 0s 42us/sample - loss: 0.0255 - mse: 0.0255
Epoch 10/100
10499/10499 [========================

10499/10499 [==============================] - 0s 35us/sample - loss: 0.0243 - mse: 0.0243
Epoch 80/100
10499/10499 [==============================] - 0s 38us/sample - loss: 0.0243 - mse: 0.0243
Epoch 81/100
10499/10499 [==============================] - 0s 34us/sample - loss: 0.0243 - mse: 0.0243
Epoch 82/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0243 - mse: 0.0243
Epoch 83/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0243 - mse: 0.0243
Epoch 84/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0243 - mse: 0.0243
Epoch 85/100
10499/10499 [==============================] - 0s 37us/sample - loss: 0.0243 - mse: 0.0243
Epoch 86/100
10499/10499 [==============================] - 0s 33us/sample - loss: 0.0243 - mse: 0.0243
Epoch 87/100
10499/10499 [==============================] - 0s 36us/sample - loss: 0.0243 - mse: 0.0243
Epoch 88/100
10499/10499 [==============================] - 0s 37us/sample - 

# Add decoder layers

In [18]:
for i in range(len(decoder_layers)-1, -1, -1):
    model.add(decoder_layers[i])
    
for layer in model.layers:
    layer.trainable = False

model.compile(loss = "mse", optimizer = "adam", metrics = ["mse"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8